# Formatting & Cleaning Stop Data

## Parsing the Stops File to Pull Information Needed for the Static Stops Table in the Database
## Writing out to a file in json format

In [159]:
import pandas as pd
import json
df_stops = pd.read_csv('stopinfo_2012.csv', encoding="utf-8")

In [160]:
df_stops.head()

,Unnamed: 0,Stop number,Name without locality,Locality,Locality number,Code,Stop No,NaPTAN stop class,NaPTANId,Easting,Northing,lat,lon
0,0,1,Pearse Street (Pearse Avenue),Sallynoggin,E0825024,->NE2,3234,BCT,8250DB003234,724091.0,726427.0,53.273638,-6.139472
1,1,1,Pearse Street (Pearse Avenue),Sallynoggin,E0825024,->SW1,3210,BCT,8250DB003210,724087.0,726412.0,53.273504,-6.139538
2,4,3,Clontarf Road Junction Strandville Avenu,Clontarf,E0822038,1,174,BCT,8220DB001741,718428.0,736222.0,53.362917,-6.220676
3,6,4,College Green,Dublin City South,E0822091,2,135,BCT,8220DB001359,715770.0,734095.0,53.344399,-6.261369
4,8,4,College Green,Dublin City South,E0822091,4,45,BCT,8220DB004522,715821.0,734081.0,53.344262,-6.260608


In [161]:
df_stops.columns

Index(['Unnamed: 0', 'Stop number', 'Name without locality', 'Locality',
       'Locality number', 'Code', 'Stop No', 'NaPTAN stop class', 'NaPTANId',
       'Easting', 'Northing', 'lat', 'lon'],
      dtype='object')

In [162]:
df_stops.drop(df_stops.columns[[0, 1, 4, 5, 6, 7, 9, 10]], axis=1, inplace=True)

In [163]:
df_stops.columns

Index(['Name without locality', 'Locality', 'NaPTANId', 'lat', 'lon'], dtype='object')

In [164]:
df_stops.isnull().sum()

Name without locality    0
Locality                 0
NaPTANId                 0
lat                      0
lon                      0
dtype: int64

In [165]:
df_stops.shape

(5255, 5)

In [166]:
class Stop:
    def __init__(self, address, stop_id, latitude, longitude):
        self.__address = address
        self.__stop_id = stop_id
        self.__latitude = latitude
        self.__longitude = longitude
    
    def __str__(self):
        str1 = "Stop ID: " + self.__stop_id + "\nStop Address: " + self.__address + "\nLatitude: " + str(self.__latitude) + "\nLongitude: " + str(self.__longitude) 
        return str1
        
stops = []

In [167]:
df_stops.head()

,Name without locality,Locality,NaPTANId,lat,lon
0,Pearse Street (Pearse Avenue),Sallynoggin,8250DB003234,53.273638,-6.139472
1,Pearse Street (Pearse Avenue),Sallynoggin,8250DB003210,53.273504,-6.139538
2,Clontarf Road Junction Strandville Avenu,Clontarf,8220DB001741,53.362917,-6.220676
3,College Green,Dublin City South,8220DB001359,53.344399,-6.261369
4,College Green,Dublin City South,8220DB004522,53.344262,-6.260608


In [168]:
stops_2012 = {}
temp_stop_id = "Null"
temp_stop_address = "Null"
year = 2012

df_stops['stop_id'] = temp_stop_id
df_stops['stop_address'] = temp_stop_address
df_stops['year'] = year

for index, row in df_stops.iterrows():
    stop_address = row['Name without locality'] + ", " + row['Locality']
    stop_id = row['NaPTANId'][8:]
    if stop_id.startswith("00"):
        stop_id = stop_id[2:]
    elif stop_id.startswith("0"):
        stop_id = stop_id[1:]
    lat = row["lat"]
    lon = row["lon"]
    stops_2012[stop_id] = {"stop_address": stop_address, "latitude": lat, "longitude": lon}
    
    df_stops.set_value(index, 'stop_id', stop_id)
    df_stops.set_value(index, 'stop_address', stop_address)

In [169]:
df_stops.head()

,Name without locality,Locality,NaPTANId,lat,lon,stop_id,stop_address,year
0,Pearse Street (Pearse Avenue),Sallynoggin,8250DB003234,53.273638,-6.139472,3234,"Pearse Street (Pearse Avenue), Sallynoggin",2012
1,Pearse Street (Pearse Avenue),Sallynoggin,8250DB003210,53.273504,-6.139538,3210,"Pearse Street (Pearse Avenue), Sallynoggin",2012
2,Clontarf Road Junction Strandville Avenu,Clontarf,8220DB001741,53.362917,-6.220676,1741,"Clontarf Road Junction Strandville Avenu, Clon...",2012
3,College Green,Dublin City South,8220DB001359,53.344399,-6.261369,1359,"College Green, Dublin City South",2012
4,College Green,Dublin City South,8220DB004522,53.344262,-6.260608,4522,"College Green, Dublin City South",2012


In [170]:
df_stops.drop(df_stops.columns[[0, 1, 2]], axis=1, inplace=True)
df_stops.columns

Index(['lat', 'lon', 'stop_id', 'stop_address', 'year'], dtype='object')

In [171]:
print(stops_2012['3234'])

{'stop_address': 'Pearse Street (Pearse Avenue), Sallynoggin', 'latitude': 53.27363813901732, 'longitude': -6.139471738315902}


In [172]:
with open('stop_objects_2012.txt', 'w') as f:
    json.dump(stops_2012, f)

# 2017 Stops

In [173]:
df_stops_2017 = pd.read_csv('2017_stops.csv', encoding="utf-8")
df_stops_2017.head()

,stop_id,stop_address,latitude,longitude
0,2,Parnell Square,53.352241,-6.263695
1,3,Parnell Square,53.352307,-6.263783
2,4,Parnell Square,53.352567,-6.264166
3,6,Parnell Square,53.352744,-6.264443
4,7,Parnell Square,53.352836,-6.264562


In [174]:
year = 2017
df_stops_2017['year'] = year

In [175]:
stops_2017 = {}
for index, row in df_stops_2017.iterrows():
    stop_id = row["stop_id"]
    stop_address = row["stop_address"]
    latitude = row["latitude"]
    longitude = row["longitude"]
    stops_2017[stop_id] = {"stop_address": stop_address, "latitude": latitude, "longitude": longitude}

In [176]:
with open('stop_info_2017_cleaned.txt', 'w') as f:
    json.dump(stops_2017, f)

In [177]:
df_stops_2017.describe().T

,count,mean,std,min,25%,50%,75%,max
stop_id,9924.0,176627.806530,211681.913285,2.000000,3006.750000,103056.000000,331423.500000,991231.000000
latitude,9924.0,53.125063,0.982737,0.000000,52.880134,53.317570,53.396683,55.002956
longitude,9924.0,-7.205049,1.175565,-10.453396,-8.460172,-6.442055,-6.256965,0.000000
year,9924.0,2017.000000,0.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000


In [178]:
df_stops_2017['latitude'] = df_stops_2017['latitude'].astype('category')
df_stops_2017['longitude'] = df_stops_2017['longitude'].astype('category')

In [179]:
categorical_features = ['longitude', 'latitude']
df_stops_2017[categorical_features].describe().T

,count,unique,top,freq
longitude,9924.0,9828.0,0.000000,2.0
latitude,9924.0,9811.0,53.277248,3.0


In [180]:
df_stops_2017.shape

(9924, 5)

In [181]:
df_stops.columns

Index(['lat', 'lon', 'stop_id', 'stop_address', 'year'], dtype='object')

In [182]:
print(df_stops.columns)
print(df_stops_2017.columns)

Index(['lat', 'lon', 'stop_id', 'stop_address', 'year'], dtype='object')
Index(['stop_id', 'stop_address', 'latitude', 'longitude', 'year'], dtype='object')


In [183]:
df_stops = df_stops.rename(columns={"lat": "latitude", "lon": "longitude"})

In [184]:
frames = [df_stops_2017, df_stops]
df_all_stops = pd.concat(frames)

In [185]:
df_all_stops.head()

,latitude,longitude,stop_address,stop_id,year
0,53.352241,-6.263695,Parnell Square,2,2017
1,53.352307,-6.263783,Parnell Square,3,2017
2,53.352567,-6.264166,Parnell Square,4,2017
3,53.352744,-6.264443,Parnell Square,6,2017
4,53.352836,-6.264562,Parnell Square,7,2017


In [186]:
print(df_stops.shape)
print(df_stops_2017.shape)

(5255, 5)
(9924, 5)


In [187]:
df_all_stops['longitude'] = df_all_stops['longitude'].astype('category')
df_all_stops['latitude'] = df_all_stops['latitude'].astype('category')
categorical_features = ['latitude', 'longitude']
df_all_stops[categorical_features].describe().T

,count,unique,top,freq
latitude,15179.0,15062.0,53.277248,3.0
longitude,15179.0,15079.0,0.000000,2.0


In [188]:
df_all_stops.shape

(15179, 5)